In [1]:
import os
import requests
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.tools import DuckDuckGoSearchResults

llm=ChatOpenAI(temperature=0.1)

alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")

class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(description="The query you will search for")

class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    Example query: Stock Market Symbol for Apple Company
    """
    args_schema: Type[StockMarketSymbolSearchToolArgsSchema] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchResults()
        return ddg.run(query)


class CompanyOverviewToolArgsSchema(BaseModel):
    symbol : str = Field(
        description="Stock symbol of the company. Example: AAPL, TSLA",
    )

class CompanyOverviewTool(BaseTool):
    name="CompanyOverview"
    description="""
    Use this to get an overview of the financials of the company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewToolArgsSchema] = CompanyOverviewToolArgsSchema

    def _run(self, symbol):
        r = requests.get(f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}")
        return r.json()


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools = [
        StockMarketSymbolSearchTool(),
        CompanyOverviewTool(),
    ],
)

# prompt = "Give me information on Cloudflare's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have."
prompt = "Give me financial information on Cloudflare's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment."

agent.invoke(prompt)




> Entering new AgentExecutor chain...


Authentication failed for https://api.smith.langchain.com/runs. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs', '{"detail":"Using legacy API key. Please generate a new API key."}')



Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Cloudflare stock'}`




Authentication failed for https://api.smith.langchain.com/runs/3068209d-0087-403e-93fe-3fcb33d7b905. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/3068209d-0087-403e-93fe-3fcb33d7b905', '{"detail":"Using legacy API key. Please generate a new API key."}')


[snippet: Get the latest information on Cloudflare Inc (NET), a software company that offers security and web performance solutions. See its stock price, fair value, news, financials, valuation, and more., title: Cloudflare Stock Price - NYSE: NET - Morningstar, link: https://www.morningstar.com/stocks/xnys/net/quote], [snippet: Get the latest information on Cloudflare, Inc. (NET), a leading connectivity cloud company, including its stock price, news, analysis and forecast. See how Cloudflare performs in the market, its financials, metrics, dividends, history and more., title: Cloudflare, Inc. (NET) Stock Price, Quote & News - Stock Analysis, link: https://stockanalysis.com/stocks/net/], [snippet: In the latest market close, Cloudflare (NET) reached $89.52, with a +1.8% movement compared to the previous day. The stock's performance was ahead of the S&P 500's daily gain of 0.16%. Meanwhile ..., title: Cloudflare (NET) Beats Stock Market Upswing: What Investors ... - Nasdaq, link: https:

{'input': "Give me information on Cloudflare's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have.",
 'output': 'Cloudflare Inc. has the stock symbol "NET" on the New York Stock Exchange (NYSE). \n\nHere are some key points about Cloudflare\'s stock:\n- In the latest market close, Cloudflare (NET) reached $89.52, with a +1.8% movement compared to the previous day.\n- Cloudflare\'s stock performance was ahead of the S&P 500\'s daily gain of 0.16%.\n- Cloudflare shares rose 9.2% after naming a former ServiceNow executive as President of Product & Engineering.\n- The market also reacted positively to the Fed\'s rate cut and the outlook for Cloudflare\'s growth.\n\nTo analyze if Cloudflare\'s stock is a potential good investment, we can look at factors such as the company\'s financial performance, growth prospects, industry trends, and market conditions. It\'s important to conduct a thorough analysis and consider your investment goal